In [107]:
import pandas as pd
train = pd.read_csv('C:\\Users\\saulpark\\Desktop\\scotch_review.csv')

train.shape

new = train['description']
print(new)

0       Magnificently powerful and intense. Caramels, ...
1       What impresses me most is how this whisky evol...
2       There have been some legendary Bowmores from t...
3       With a name inspired by a 1926 Buster Keaton m...
4       Captivating, enticing, and wonderfully charmin...
5       Powerful, muscular, well-textured, and invigor...
6       Deep gold color. Surprisingly lively on the no...
7       Definitely showing its age, but not in a bad w...
8       The Dalmore is one of a handful of whiskies th...
9       A rich amber color and elegantly oxidized note...
10      It’s great that Glenlivet releases whiskies un...
11      Classic sherry cask-aged Macallan. Antique amb...
12      A remarkable beauty from the Angus town of Mon...
13      A marriage of three different single malts, ag...
14      As you’d expect, solid peat is the first thing...
15      Formulated to celebrate the 200th anniversary ...
16      An essay in balance on both the aroma and pala...
17      Cask #

In [108]:
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.snowball import SnowballStemmer

def review_to_words( raw_review ):
    # 1. HTML 제거
    review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
    # 2. 영문자가 아닌 문자는 공백으로 변환
    letters_only = re.sub('[^a-zA-Z]', ' ', review_text)
    # 3. 소문자 변환
    words = letters_only.lower().split()
    # 4. 파이썬에서는 리스트보다 세트로 찾는게 훨씬 빠르다.
    # stopwords 를 세트로 변환한다.
    stops = set(stopwords.words('english'))
    # 5. Stopwords 불용어 제거
    meaningful_words = [w for w in words if not w in stops]
    # 6. 어간추출
    stemming_words = [stemmer.stem(w) for w in meaningful_words]
    # 7. 공백으로 구분된 문자열로 결합하여 결과를 반환
    return( ' '.join(stemming_words) )

In [118]:

new_description = pd.Series()
stemmer = SnowballStemmer('english')
#print(type(desc_train))
for idx, row in desc_train.iteritems():
    clean_review = review_to_words(train['description'][idx])
    desc_train[idx] = clean_review
    new_description.append(pd.Series([clean_review]))

print(desc_train)
train['description'] = desc_train
#print(new_description)
train.to_csv('review_preprocessed.csv')

0       magnif power inten caramel dri peat eleg cigar...
1       impress whiski evolv incr complex nose palat t...
2       legendari bowmor mid everi bit equal share rem...
3       name inspir buster keaton movi bottl produc ne...
4       captiv entic wonder charm first blend malt chi...
5       power muscular well textur invigor even within...
6       deep gold color surpri live nose age complex a...
7       definit show age bad way distilleri charact st...
8       dalmor one hand whiski seem abl age cask mani ...
9       rich amber color eleg oxid note greet luscious...
10      great glenlivet relea whiski cellar collect la...
11      classic sherri cask age macallan antiqu amber ...
12      remark beauti angus town montro eleg nose show...
13      marriag three differ singl malt age american f...
14      expect solid peat first thing glass peat beast...
15      formul celebr th anniversari johnni walker fab...
16      essay balanc aroma palat silki layer delic swe...
17      cask l

In [146]:
desc_train.to_csv('desc_text.txt', header=None, index=None, sep=' ', mode='a')

In [151]:
def remove_newlines(fname):
    flist = open('desc_text.txt').readlines()
    flist = [s.rstrip('"') for s in flist]
    flist = [s.replace('"', '') for s in flist]
    print(flist)
    file = open('desc_text.txt','w')
    file.write(''.join(flist))

In [152]:
remove_newlines("")

['magnif power inten caramel dri peat eleg cigar smoke seed scrape vanilla bean brand new pencil peppercorn coriand seed star ani make deepli satisfi nose experi silki caramel bounti fruit ripe peach stew appl orang pith perva smoke element burnt tobacco abid finish smoke dri spice banoff pie sweet close perfect editor choic\n', 'impress whiski evolv incr complex nose palat thick viscous whiski note sticki toff earthi oak fig cake roast nut fallen fruit pancak batter black cherri ripe peach dark chocol cover espresso bean polish leather tobacco hint wild game linger leafi damp kiln smoke flavor continu palat long swallow hope dream older whiski\n', 'legendari bowmor mid everi bit equal share remark aroma tropic fruit move hallucinatori inten guava mango peach pineappl grapefruit light touch peat smoke memori islay realiti concentr even low strength palat silki headi haunt last forev dri glass legend born eight bottl u editor choic\n', 'name inspir buster keaton movi bottl produc news o

In [130]:
remove_newlines("")

['magnif power inten caramel dri peat eleg cigar smoke seed scrape vanilla bean brand new pencil peppercorn coriand seed star ani make deepli satisfi nose experi silki caramel bounti fruit ripe peach stew appl orang pith perva smoke element burnt tobacco abid finish smoke dri spice banoff pie sweet close perfect editor choic"\n', 'impress whiski evolv incr complex nose palat thick viscous whiski note sticki toff earthi oak fig cake roast nut fallen fruit pancak batter black cherri ripe peach dark chocol cover espresso bean polish leather tobacco hint wild game linger leafi damp kiln smoke flavor continu palat long swallow hope dream older whiski"\n', 'legendari bowmor mid everi bit equal share remark aroma tropic fruit move hallucinatori inten guava mango peach pineappl grapefruit light touch peat smoke memori islay realiti concentr even low strength palat silki headi haunt last forev dri glass legend born eight bottl u editor choic"\n', 'name inspir buster keaton movi bottl produc new

In [24]:
flist = open('desc_text.txt').readlines()

desc_split=list()

[desc_split.append(s.split(" ")) for s in flist]
desc_split

[['magnif',
  'power',
  'inten',
  'caramel',
  'dri',
  'peat',
  'eleg',
  'cigar',
  'smoke',
  'seed',
  'scrape',
  'vanilla',
  'bean',
  'brand',
  'new',
  'pencil',
  'peppercorn',
  'coriand',
  'seed',
  'star',
  'ani',
  'make',
  'deepli',
  'satisfi',
  'nose',
  'experi',
  'silki',
  'caramel',
  'bounti',
  'fruit',
  'ripe',
  'peach',
  'stew',
  'appl',
  'orang',
  'pith',
  'perva',
  'smoke',
  'element',
  'burnt',
  'tobacco',
  'abid',
  'finish',
  'smoke',
  'dri',
  'spice',
  'banoff',
  'pie',
  'sweet',
  'close',
  'perfect',
  'editor',
  'choic\n'],
 ['impress',
  'whiski',
  'evolv',
  'incr',
  'complex',
  'nose',
  'palat',
  'thick',
  'viscous',
  'whiski',
  'note',
  'sticki',
  'toff',
  'earthi',
  'oak',
  'fig',
  'cake',
  'roast',
  'nut',
  'fallen',
  'fruit',
  'pancak',
  'batter',
  'black',
  'cherri',
  'ripe',
  'peach',
  'dark',
  'chocol',
  'cover',
  'espresso',
  'bean',
  'polish',
  'leather',
  'tobacco',
  'hint',
  '